## Context 

raw features capture the full predictive signal in the data. FE transfroms domain knowledge into model inputs. creating new perspectives on the data that can imporve model performance significanly

## objective 

- create time-derived features from TransactionDT
- build card-level based on address level aggregation features
- engginer email domain and interaction features
-  document transformation logic for profuction reproducibility

In [1]:
# import and load data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

train = pd.read_parquet(Path('../data/interim/train_merged.parquet'))
print(f'Data loaded: {train.shape}')

Data loaded: (590540, 434)
